# Topic Modelling

<style>
    div.output_area{
        max-height:600px;
        overflow:scroll;
    }
</style>

<h3>Packages</h3>

In [1]:
import os, json, gensim, nltk
from nltk import word_tokenize
from collections import Counter, defaultdict
import matplotlib.pyplot as plt

<h3> Funktionen zum Einlesen von Dateien </h3>

In [2]:
def loadjson(filename):
    import json
    with open((filename+'.json'), 'r') as f:
        file = json.loads(f.read())
    print(len(file))
    return file

def savejson(file, filename):
    with open((filename+'.json'), 'w') as f:
        json.dump(file, f)
    print(True)

def read_lines(txtfile):
    with open (txtfile, 'r') as f:
        text = f.read().split('\n')
        newfile = []
        for line in text:
            newfile += line.split(' ')
        newfile = sorted(list(set(newfile)))
        if '' in newfile:
            newfile = newfile[1:]
    print(len(newfile))
    return newfile

<h3>Funktionen zur Korpus- und Topicanalyse</h3>

In [3]:
def count_types(text):
    counter = Counter(text[0])
    for i in range(1,len(text)):
        counter.update(text[i])
    return counter

def print_topics(model, dictionary, num_topics, num_terms):
    for i in list(range(num_topics)):
        print('Topic ' + str(i+1) + ':')
        for wordid, prob in model.get_topic_terms(i, num_terms):
            print(dictionary[wordid], prob)
        print(' ')

# Einlesen der Textdateien

Der folgende Code dient dazu, die Untertitel-Textdateien in Python einzulesen und als String-Liste zu speichern. Parallel dazu wird eine Liste von Dateipfaden erstellt, die es erlaubt, einen Film bei Bedarf anhand seines Index wiederzuerkennen und zu öffnen. Die Ausgabe zeigt die Anzahl der enthaltenen Filme pro Halbdekade, wobei deutlich wird, dass die Datenlage in den Jahren vor 1925 und in den Nachkriegsjahren 1945-1950 sehr spärlich ist.

In [4]:
dirname = 'subtitleData'

In [5]:
texts = []
filenames = []
filmcount = {}
for folder in sorted(os.listdir(dirname)):
    if not folder.startswith('.') and folder != 'movieDB.json':
        folderpath='/'.join([dirname,folder])
        print(folder, ':', len(os.listdir(folderpath)))
        filmcount[folder]=len(os.listdir(folderpath))
        for file in sorted(os.listdir(folderpath)):
            if not file.startswith('.'):
                filepath='/'.join([folderpath,file])
                with open(filepath, 'r') as f:
                     texts.append(f.read())
                filenames.append(filepath)
len(texts)

1900_1905 : 1
1910_1915 : 1
1915_1920 : 2
1920_1925 : 3
1925_1930 : 19
1930_1935 : 51
1935_1940 : 48
1940_1945 : 27
1945_1950 : 9
1950_1955 : 123
1955_1960 : 218
1960_1965 : 229
1965_1970 : 516
1970_1975 : 425
1975_1980 : 847
1980_1985 : 1108
1985_1990 : 1089
1990_1995 : 1292
1995_2000 : 3722
2000_2005 : 3681
2005_2010 : 5778
2010_2015 : 8039
2015_2020 : 5998


33226

Beispiel einer eingelesenen Untertitel-Datei:

In [19]:
texts[0][:2068]

' A Trip to the Moon by George Melies The astronomers are assembled in a large hall embellished with instruments . The president and members of the committee enter . Everybody takes his seat . Entrance of six man-servants carrying the telescopes of the astronomers . The president takes his chair and explains to the members his plan for a trip to the moon . The scheme is approved by many . But one member violently opposes same . After some argument , the president throws his papers and books at his head . Upon order being restored , the trip proposed by the president is voted by acclimation . The man-servants bring travelling suits . President Barbenfouillis selects five colleagues to accompany him : Nostradamus , Alcofrisbas , Omega , Micromegas and Parafaragamus . We enter the interior of the workshop where smiths , mechanics , weighers , carpenters , upholsterers , et cetera are working hard at the completion of the machine . Micromegas accidentally falls into a tub of nitric acid . 

Im nächsten Schritt werden die Texte mithilfe der NLTK-Funktion <code>word_tokenize()</code> in Listen von Wörtern und Satzzeichen umgewandelt, wobei Satzzeichen sowie apostrophierte Wörter als eigene Tokens gelten.

In [9]:
tokenized = [word_tokenize(text) for text in texts]
len(tokenized)

33226


Auszug aus dem obigen Text in tokenisierter Form:

In [11]:
tokenized[0][:45]

['A',
 'Trip',
 'to',
 'the',
 'Moon',
 'by',
 'George',
 'Melies',
 'The',
 'astronomers',
 'are',
 'assembled',
 'in',
 'a',
 'large',
 'hall',
 'embellished',
 'with',
 'instruments',
 '.',
 'The',
 'president',
 'and',
 'members',
 'of',
 'the',
 'committee',
 'enter',
 '.',
 'Everybody',
 'takes',
 'his',
 'seat',
 '.',
 'Entrance',
 'of',
 'six',
 'man-servants',
 'carrying',
 'the',
 'telescopes',
 'of',
 'the',
 'astronomers',
 '.']

<h3> Eliminierung grundlegeneder Stoppwörter </h3>

In einem ersten vorbereitenden Schritt wird die von NLTK bereitgestellte englische Stoppwortliste verwendet, um häufige Funktionswörter wie Partikel, Pronomen und Hilfsverben aus dem Korpus zu entfernen. Die resultierende Datenstruktur wird zur Weiterverarbeitung als json-File gespeichert.

In [20]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
len(stop_words)

179

In [21]:
stop_words[:20]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his']

In [14]:
clean = [[w.lower() for w in doc if w.isalpha() and len(w)>2 and w not in stop_words] for doc in tokenized]
len(clean)

33226


33226

Auszug aus dem obigen Text nach Entfernung der Stoppwörter:

In [15]:
clean[0][:22]

['trip',
 'moon',
 'george',
 'melies',
 'astronomers',
 'assembled',
 'large',
 'hall',
 'embellished',
 'instruments',
 'president',
 'members',
 'committee',
 'enter',
 'everybody',
 'takes',
 'seat',
 'entrance',
 'six',
 'carrying',
 'telescopes',
 'astronomers']

In [16]:
savejson(clean, 'clean')

True

In [6]:
clean = loadjson('../clean') #todelete

33226


<h3>Wortfrequenzanalyse</h3>

Unter Verwendung der Klasse Counter werden die 1000 häufigsten Types mit ihrer jeweiligen Häufigkeit ausgegeben.

In [7]:
wordcount = count_types(clean)

In [11]:
for k,v in list(enumerate(wordcount.most_common(500),1)):
    print(k,v)

1 ('know', 920105)
2 ('get', 771727)
3 ('right', 662235)
4 ('like', 599955)
5 ('got', 569668)
6 ('one', 560013)
7 ('come', 536680)
8 ('yeah', 455627)
9 ('let', 431389)
10 ('well', 428217)
11 ('think', 423863)
12 ('okay', 419240)
13 ('want', 404875)
14 ('see', 398845)
15 ('back', 396403)
16 ('going', 370479)
17 ('time', 356817)
18 ('good', 344284)
19 ('gon', 340664)
20 ('need', 328989)
21 ('look', 319648)
22 ('take', 310415)
23 ('yes', 304441)
24 ('man', 297660)
25 ('would', 294610)
26 ('could', 292649)
27 ('hey', 285842)
28 ('something', 283752)
29 ('way', 269660)
30 ('tell', 257416)
31 ('people', 238389)
32 ('make', 229671)
33 ('say', 214979)
34 ('please', 214433)
35 ('really', 208498)
36 ('never', 208012)
37 ('help', 207305)
38 ('find', 197655)
39 ('sir', 194362)
40 ('sorry', 189520)
41 ('thing', 187814)
42 ('two', 186760)
43 ('little', 174628)
44 ('mean', 174485)
45 ('god', 171040)
46 ('give', 168036)
47 ('sure', 166173)
48 ('still', 164483)
49 ('said', 161771)
50 ('stop', 160385)
5

<h3>Erstellung des Wörterbuchs und Dimensionalitätsreduktion</h3>

Das Wörterbuch wird auf Grundlage der Klasse Dictionary erstellt. 

In [26]:
dictionary = gensim.corpora.Dictionary(clean)
print(len(dictionary))

198918


Um orthographische Fehler, seltene Eigennamen und andere spezifische Begriffe zu entfernen, wird die Untergenze für die Dokumenthäufigkeit auf 100 gesetzt. Die Größe des Wörterbuchs wird dabei um fast 90% reduziert.

In [27]:
dictionary.filter_extremes(no_below=100,no_above=1)
print(len(dictionary))

21245


Andererseits werden alle Types herausgefiltert, die in mehr als 35% der Dokumente vorkommen. Die Liste <code>removed</code> zeigt, welche 503 Begriffe dadurch aus dem Wörterbuch entfernt worden sind.

In [30]:
tokens = list(dictionary.values())
dictionary.filter_extremes(no_below=100,no_above=.35)
len(dictionary)

20742

In [31]:
removed = set(tokens)-set(dictionary.values())
removed

{'able',
 'actually',
 'afraid',
 'ago',
 'ahead',
 'air',
 'alive',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'always',
 'another',
 'answer',
 'anybody',
 'anymore',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'area',
 'around',
 'ask',
 'asked',
 'asking',
 'ass',
 'attention',
 'away',
 'baby',
 'back',
 'bad',
 'beautiful',
 'become',
 'behind',
 'believe',
 'best',
 'better',
 'big',
 'bit',
 'blood',
 'body',
 'boy',
 'brain',
 'break',
 'bring',
 'brother',
 'brought',
 'building',
 'business',
 'call',
 'called',
 'calling',
 'came',
 'car',
 'care',
 'careful',
 'case',
 'catch',
 'cause',
 'chance',
 'change',
 'changed',
 'check',
 'child',
 'choice',
 'city',
 'clean',
 'clear',
 'close',
 'come',
 'comes',
 'coming',
 'completely',
 'contact',
 'control',
 'could',
 'couple',
 'course',
 'cover',
 'crazy',
 'cut',
 'dad',
 'damn',
 'dangerous',
 'day',
 'days',
 'dead',
 'deal',
 'death',
 'die',
 'died',
 'different',
 'doctor',
 'done',
 'door',
 'drink'

In [20]:
len(removed)

503

Die Liste der in mehr als 35% aller Dokumente auftretenden Wörter ist mit der Liste der häufigsten Types zu großen Teilen, aber nicht vollständig deckungsgleich:

In [43]:
wrdlst = [x[0] for x in wordcount.most_common(1000)]
sorted([(wrdlst.index(word)+1, word) for word in removed], key=lambda x:x[0])

[(1, 'know'),
 (2, 'get'),
 (3, 'right'),
 (4, 'like'),
 (5, 'got'),
 (6, 'one'),
 (7, 'come'),
 (8, 'yeah'),
 (9, 'let'),
 (10, 'well'),
 (11, 'think'),
 (12, 'okay'),
 (13, 'want'),
 (14, 'see'),
 (15, 'back'),
 (16, 'going'),
 (17, 'time'),
 (18, 'good'),
 (19, 'gon'),
 (20, 'need'),
 (21, 'look'),
 (22, 'take'),
 (23, 'yes'),
 (24, 'man'),
 (25, 'would'),
 (26, 'could'),
 (27, 'hey'),
 (28, 'something'),
 (29, 'way'),
 (30, 'tell'),
 (31, 'people'),
 (32, 'make'),
 (33, 'say'),
 (34, 'please'),
 (35, 'really'),
 (36, 'never'),
 (37, 'help'),
 (38, 'find'),
 (39, 'sir'),
 (40, 'sorry'),
 (41, 'thing'),
 (42, 'two'),
 (43, 'little'),
 (44, 'mean'),
 (45, 'god'),
 (46, 'give'),
 (47, 'sure'),
 (48, 'still'),
 (49, 'said'),
 (50, 'stop'),
 (51, 'anything'),
 (52, 'even'),
 (53, 'maybe'),
 (54, 'wait'),
 (55, 'thank'),
 (56, 'much'),
 (57, 'nothing'),
 (58, 'first'),
 (59, 'life'),
 (60, 'work'),
 (61, 'keep'),
 (62, 'must'),
 (63, 'everything'),
 (64, 'put'),
 (65, 'away'),
 (66, 'worl

Um weitere Eigennamen und Partikel zu entfernen, die bisher nicht erfasst wurden, werden manuell erstellte Wortlisten verwendet. Die Namensliste basiert auf den 1000 häufigsten englischen Namen und wurde nach Erstellung eines ersten Topic Models durch weitere scifi-spezifische Namen ergänzt:

In [44]:
particles = read_lines('final/particles.txt')
namelist = loadjson('final/namelist')

309
2440


Diejenigen Namen und Partikel, die noch im Wörterbuch vorkommen, werden zu einer Stoppwortliste zusammengefasst und anhand ihrer IDs aus dem Wörterbuch entfernt.

In [45]:
scifinames = [name for name in namelist if name in dictionary.token2id]
part = [word for word in particles if word in dictionary.token2id]
stopwords=scifinames+part
bad_ids=[dictionary.token2id[word] for word in stopwords]

In [46]:
len(stopwords)

1228

In [25]:
dictionary.filter_tokens(bad_ids=bad_ids)
len(dictionary)

19515

<h3>Numerisches Korpus und Topic Model</h3>

Im letzten Schritt wird das Textkorpus in ein numerisches Bag-of-Words-Korpus umgewandelt, das für jedes Dokument die Wort-IDs und Häufigkeiten der enthaltenen Types speichert.

In [47]:
corpus = [dictionary.doc2bow(doc) for doc in clean]
len(corpus)

33226

Das LDA-Topic-Model wird mit einer willkürlich festgelegten Anzahl von 23 Topics erstellt. Die Einstellung der Parameter <code>alpha</code> und <code>eta</code> auf <code>'auto'</code> führt zu einer asymmetrischen Verteilung.

In [28]:
model = gensim.models.LdaModel(corpus,23,passes=10,alpha='auto',eta='auto')

Die 50 wahrscheinlichsten Begriffe pro Topic:

In [ ]:
print_topics(model=model, dictionary=dictionary, num_topics=23, num_terms=50)

Erstellung der Visualisierung mit pyLDAvis:

In [9]:
import pyLDAvis, pyLDAvis.gensim as ldavis
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings('ignore')

In [10]:
vis=ldavis.prepare(model,corpus,dictionary,sort_topics=False)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0      0.024710 -0.029591       1        1  3.787576
1     -0.176714  0.006331       2        1  8.366076
2     -0.077924  0.094472       3        1  6.148406
3     -0.074949  0.008784       4        1  1.279079
4     -0.160964 -0.106145       5        1  1.107579
5      0.109180 -0.125987       6        1  3.058729
6     -0.121567 -0.077132       7        1  2.832227
7     -0.037572 -0.061425       8        1  5.519195
8     -0.099543  0.107421       9        1  3.340240
9     -0.132619 -0.025711      10        1  3.200472
10     0.062790  0.097053      11        1  4.617491
11     0.064807  0.144506      12        1  4.133460
12     0.244659 -0.094301      13        1  1.145759
13    -0.057145  0.121687      14        1  7.808692
14     0.097218 -0.087952      15        1  3.123716
15    -0.013915  0.041342      16        1  1.481002
16     0.235504  0.001578      17        1  4.673467
17     0.172783 -0.095452      18        1  3.884388
18    -0.104534  0.103791      19        1  6.563064
19     0.030646  0.061589      20        1  7.141876
20    -0.137593 -0.267907      21        1  4.678469
21     0.095148  0.093432      22        1  6.639730
22     0.057594  0.089619      23        1  5.469307, topic_info=         Term           Freq          Total Category  logprob  loglift
1309    music   62577.000000   62577.000000  Default  30.0000  30.0000
3013  captain   79651.000000   79651.000000  Default  29.0000  29.0000
3191     ship  102348.000000  102348.000000  Default  28.0000  28.0000
3439    sighs   39276.000000   39276.000000  Default  27.0000  27.0000
1151    agent   47521.000000   47521.000000  Default  26.0000  26.0000
...       ...            ...            ...      ...      ...      ...
587     chief    3931.584275   15889.233135  Topic23  -5.9604   1.5094
899     group    3780.762755   16368.656132  Topic23  -5.9995   1.4406
5985   threat    3458.433456   11464.097092  Topic23  -6.0886   1.7076
242      join    3663.216659   16095.445597  Topic23  -6.0311   1.4258
549      pain    3681.278311   23857.661774  Topic23  -6.0262   1.0372

[1710 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
14749      7  0.996063      abdul
1144      11  0.035720  abilities
1144      12  0.033851  abilities
1144      13  0.018691  abilities
1144      14  0.012876  abilities
...      ...       ...        ...
9219      10  0.833068    zombies
9219      11  0.079340    zombies
9219      14  0.000696    zombies
9219      19  0.038278    zombies
9219      22  0.046630    zombies

[10523 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])

Speichern der Daten:

In [12]:
dictionary.save('final/dictionary1')
model.save('final/model1_23topics')
pyLDAvis.save_html(vis, 'final/model1_23topics.html')

<h3>Zweiter Versuch</h3>

In [ ]:
dictionary.filter_extremes(no_below=500)
print(len(dictionary))

In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in clean]
len(corpus)

In [ ]:
model = gensim.models.LdaModel(corpus,30,passes=10,alpha='auto',eta='auto')

In [ ]:
print_topics(model, dictionary, 30, 50)

In [15]:
vis=ldavis.prepare(model,corpus,dictionary,sort_topics=False)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0     -0.080510 -0.079452       1        1  2.939707
1      0.196243  0.058638       2        1  6.551517
2      0.005748 -0.040446       3        1  1.598031
3     -0.086681  0.146730       4        1  2.326492
4      0.030848 -0.080322       5        1  5.437914
5     -0.001888  0.055357       6        1  2.907075
6      0.142739  0.094453       7        1  2.259336
7     -0.243947  0.009186       8        1  2.620267
8     -0.108872  0.095003       9        1  2.999290
9      0.063721 -0.127191      10        1  4.474926
10     0.001023  0.090542      11        1  1.190017
11     0.130903 -0.113698      12        1  5.158770
12     0.084255 -0.085978      13        1  1.478364
13    -0.092429 -0.089259      14        1  4.232080
14    -0.090383  0.002354      15        1  3.486503
15     0.112118  0.190540      16        1  4.862174
16     0.103969  0.092668      17        1  3.578391
17    -0.067685 -0.028477      18        1  4.516077
18    -0.066909 -0.168565      19        1  3.160621
19    -0.153051  0.144099      20        1  2.522058
20    -0.139552 -0.075812      21        1  2.169090
21     0.038762 -0.175225      22        1  5.184020
22     0.118868 -0.078874      23        1  3.270199
23     0.007397  0.007497      24        1  1.426133
24     0.042754  0.052774      25        1  1.978191
25     0.055529  0.096870      26        1  1.778211
26    -0.063591  0.112889      27        1  1.527345
27     0.126588  0.003429      28        1  2.624289
28     0.109798 -0.062893      29        1  7.550521
29    -0.175762 -0.046836      30        1  4.192391, topic_info=          Term          Freq         Total Category  logprob  loglift
2302   captain  90122.000000  90122.000000  Default  30.0000  30.0000
2434      ship  96315.000000  96315.000000  Default  29.0000  29.0000
997      agent  55716.000000  55716.000000  Default  28.0000  28.0000
2625     sighs  38279.000000  38279.000000  Default  27.0000  27.0000
2557    grunts  27976.000000  27976.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
18      bridge   3193.114654  17402.553043  Topic30  -5.7614   1.4763
1192   support   2622.937750  13066.997637  Topic30  -5.9581   1.5661
980    weapons   2885.733287  26302.621539  Topic30  -5.8626   0.9620
143     signal   2725.870820  21689.698924  Topic30  -5.9196   1.0978
703   computer   2593.304590  23178.205304  Topic30  -5.9695   0.9816

[1734 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
990       1  0.001186  abilities
990       6  0.013281  abilities
990       8  0.045061  abilities
990      13  0.068065  abilities
990      18  0.021582  abilities
...     ...       ...        ...
3294     13  0.000420        zoo
3294     23  0.050460        zoo
3294     24  0.027753        zoo
3294     25  0.649248        zoo
3294     29  0.044993        zoo

[17681 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])

In [ ]:
dictionary.save('final/dictionary2')
model.save('final/model2_30topics')
pyLDAvis.save_html(vis, 'final/model2_30topics.html')